In [ ]:
# data manipulation
import pandas as pd
import numpy as np
from datetime import timezone, timedelta, time, datetime
from math import sqrt

# stats liibs
from scipy import stats
from scipy.stats import pearsonr
from matplotlib.projections import process_projection_requirements

# data viz
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

# apply some cool styling
plt.style.use("seaborn-darkgrid")
rcParams['figure.figsize'] = (12,  6)
sns.set(font_scale=1.5)

# make reproducible
np.random.seed(42)

# Creating Categorical Ordered
from pandas.api.types import CategoricalDtype
month_type = CategoricalDtype(categories=['Jan', 'Feb','Mar', 'Apr','May', 'Jun', 'Jul', 'Aug', 'Sep','Oct', 'Nov', 'Dec'], ordered=True)
day_type = CategoricalDtype(categories=['Monday','Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], ordered=True)

# dataset
wildfire = 'https://bit.ly/3fVhoGZ'

# Portugal Districts
# Center   = ['Castelo Branco', 'Viseu', 'Guarda', 'Aveiro', 'Coimbra', 'Leiria']
# Algarve  = ['Faro']
# Alentejo = ['Évora', 'Beja', 'Portalegre', 'Santarém']
# Lisbon   = ['Lisboa', 'Setúbal']
# Northen  = ['Viana Do Castelo', 'Porto', 'Braga', 'Vila Real', 'Bragança']

North    = ['Porto', 'Vila Real']

# Six-Step Statistical Investigation Method

* Define|Ask a Research Question
* Design a study and collect Data (Step will be skipped)
* Explore the Data
* Draw inferences
* Formulate Conclusions
* Look back & ahead

## 1. Define|Ask a Research Question

### Do the Porto and Vila Real Districts have the same characteristics regarding Fire Typology, Fire Causes, Temporal Incidence (Weekday and Month), Fire Duration and Burnt Area?

### Variables to be used: 

    * Qualitative Variables (Categorical data): 
        - Tipo: wildfire type: “Florestal, Agrícola, Falso Alarme, Queimada” 
        - Distrito: Administrative areas 
        - TipoCausa: Source – “Intencional, Natural, Negligente, Desconhecida, Reacendimento, NULL” 

    * Quantitative Variables (Numerical data) 
        - DataAlerta: Communication date 
        - HoraAlerta: Communication hour 
        - DataExtinção: Extinction date 
        - HoraExtinção: Extinction hour 
        - AA_Total (pov+mato+agric) (ha): Total burned area 



## 2. Explore the Data (Data Cleaning & Filtering)

  - Remove duplicate rows
  - Remove rows where type of wildfire is "Falso Alarme"
  - Remove rows where date of Alert is greater that date of Extinction

In [44]:
# Load dataset
df = pd.read_csv(wildfire, sep=';')
df.head()

,Ano,Codigo SGIF,Codigo_ANPC,Tipo,Distrito,Concelho,Freguesia,Local,INE,x,...,Falso Alarme,Fogacho,Incendio,Agricola,Perimetro,APS,Causa,TipoCausa,Região PROF,UGF
0,2015,DM315252,2015160007544,Florestal,Viana do Castelo,Ponte de Lima,Serdedelo,VALE DE TROVELA (SERDEDELO),160746,167814,...,0,0,1,0,NaN,NaN,122.0,Negligente,Entre Douro e Minho,Alto Minho
1,2015,DM215305,2015130043758,Florestal,Porto,Marco de Canaveses,Vila Boa de Quires,LUGAR DO FOFO,130731,193731,...,0,0,1,0,NaN,NaN,122.0,Negligente,Entre Douro e Minho,Tâmega
2,2015,DM415293,2015030021973,Florestal,Braga,Vila Verde,Lage,RUA PROF ABEL MADEIRA,31323,172162,...,0,1,0,0,NaN,NaN,60.0,Desconhecida,Entre Douro e Minho,Baixo Minho
3,2015,DM315261,2015160007563,Falso Alarme,Viana do Castelo,Ponte de Lima,Boalhosa,J. DEPOSITOS DE AGUA (BOALHOSA),160709,171291,...,1,0,0,0,NaN,NaN,630.0,Desconhecida,Entre Douro e Minho,Alto Minho
4,2015,BL115321,2015180015874,Agrícola,Viseu,Castro Daire,Castro Daire,Farejinhas - Bº das Queirós,180304,217299,...,0,1,0,0,NaN,NaN,630.0,Desconhecida,Beira Litoral,Dão Lafões


In [45]:
# Inspecting dataset colunms name
df.columns

Index(['Ano', 'Codigo SGIF', 'Codigo_ANPC', 'Tipo', 'Distrito', 'Concelho',
       'Freguesia', 'Local', 'INE', 'x', 'y', 'lat', 'lon', 'DataAlerta',
       'HoraAlerta', 'DataExtincao', 'HoraExtincao', 'Data1Intervencao',
       'Hora1Intervencao', 'FonteAlerta', 'NUT', 'AA_Povoamento (ha)',
       'AA_Mato (ha)', 'AA_Agricola (ha)',
       'AA_EspacosFlorestais (pov+mato)(ha)', 'AA_Total (pov+mato+agric) (ha)',
       'Reacendimentos', 'Queimada', 'Falso Alarme', 'Fogacho', 'Incendio',
       'Agricola', 'Perimetro', 'APS', 'Causa', 'TipoCausa', 'Região PROF',
       'UGF'],
      dtype='object')

In [46]:
# Inspecting dataset shape
print(f'The dataset has {df.shape[0]} rows and {df.shape[1]} columns. ')

The dataset has 23175 rows and 38 columns. 


In [54]:
# Inspect duplicate rows
print(f'Number of Duplicated Rows: {df.duplicated().sum()}')

Number of Duplicated Rows: 0


In [55]:
# Inspecting colunms with null values
display(df[df.columns[df.isnull().any()]].isnull().sum(axis=0))

Local                   1
DataExtincao          312
HoraExtincao          317
Data1Intervencao     1199
Hora1Intervencao     1213
FonteAlerta         23175
Perimetro           22349
APS                 22116
Causa                7013
TipoCausa            7013
Região PROF             2
UGF                     1
dtype: int64

In [56]:
# Inspecting columns datatype
df[['DataAlerta','HoraAlerta', 'DataExtincao', 'HoraExtincao']].dtypes

DataAlerta      object
HoraAlerta      object
DataExtincao    object
HoraExtincao    object
dtype: object

In [58]:
# Converting Date Columens to datetime
df['DAlert'] = df['DataAlerta'].str[:10].astype(str) + ' ' + df['HoraAlerta'].astype(str)
df['DExtiction'] = df['DataExtincao'].str[:10].astype(str) + ' ' + df['HoraExtincao'].astype(str)

df[['DAlert', 'DExtiction']] = df[['DAlert', 'DExtiction']].apply(pd.to_datetime, errors='coerce')

# Inspecting columns datatype
print(df[['DAlert', 'DExtiction']].dtypes)

# Inspecting first rows
df[['DAlert', 'DExtiction']].head(2)

DAlert        datetime64[ns]
DExtiction    datetime64[ns]
dtype: object


,DAlert,DExtiction
0,2015-03-24 17:01:00,2015-03-24 18:09:00
1,2015-03-24 17:10:00,2015-03-24 18:47:00


In [61]:
print('Number of wildfires by Type - Absolute')
display(df.Tipo.value_counts(dropna=False))

print('Number of wildfires by Type - Relative')
display(df.Tipo.value_counts(dropna=False, normalize=True))

Number of wildfires by Type - Absolute


Florestal       15464
Agrícola         4163
Falso Alarme     2925
Queimada          623
Name: Tipo, dtype: int64

Number of wildfires by Type - Relative


Florestal       0.667271
Agrícola        0.179633
Falso Alarme    0.126214
Queimada        0.026882
Name: Tipo, dtype: float64

In [80]:
# Columns for analysis
columns = ['Distrito','Tipo', 'TipoCausa', 'DAlert','DExtiction','AA_Total (pov+mato+agric) (ha)']
names   = ['District','Type', 'TypeCause', 'DateAlert', 'DateExtinction','AreaTotal']

# Filter the dataset
filtered = df[columns].query('Distrito in @North & Tipo != "Falso Alarme"')

# Reset & drop Indexes
filtered.reset_index(inplace=True, drop=True)

# Renaming columns
filtered.columns = names
filtered.head()

,District,Type,TypeCause,DateAlert,DateExtinction,AreaTotal
0,Porto,Florestal,Negligente,2015-03-24 17:10:00,2015-03-24 18:47:00,"1,35"
1,Vila Real,Florestal,Negligente,2015-03-24 21:40:00,2015-03-25 05:45:00,"38,5"
2,Porto,Florestal,NaN,2015-03-25 15:30:00,2015-03-25 18:25:00,"0,05"
3,Vila Real,Florestal,Negligente,2015-03-25 16:00:00,2015-03-25 17:00:00,"0,01"
4,Vila Real,Florestal,Negligente,2015-03-25 16:40:00,2015-03-25 18:00:00,"0,02"


In [81]:
# Inspecting Type of Fire
display(filtered.Type.value_counts(normalize=True))

Florestal    0.950225
Agrícola     0.046582
Queimada     0.003193
Name: Type, dtype: float64

In [82]:
# Calculating time of fire and getting day of week

filtered['FireDuration']   = filtered.DateExtinction - filtered.DateAlert
filtered['DayAlert']       = filtered.DateAlert.dt.day_name()
filtered['Month']          = filtered.DateExtinction.dt.month_name().str[:3]
filtered['HourTotal']      = filtered.FireDuration / pd.Timedelta(1, 'h')
filtered['DayOfYear']      = filtered.DateAlert.dt.day_of_year
filtered['HourOfDay']      = filtered.DateAlert.dt.hour

# Converting datatypes
filtered['AreaTotal'] = filtered['AreaTotal'].str.replace(',', '.').astype('float')
filtered['Month'] = filtered['Month'].astype(month_type)
filtered['DayAlert'] = filtered['DayAlert'].astype(day_type)



In [83]:
# Inspecting colunms with null values
display(filtered[filtered.columns[filtered.isnull().any()]].isnull().sum(axis=0))

TypeCause         2081
DateExtinction       9
FireDuration         9
Month                9
HourTotal            9
dtype: int64

In [84]:
# Rows with null values on Date Extinction
display(filtered[filtered.Month.isnull()])

# Removing rows with null values on Date Extinction
filtered.dropna(subset=['Month'], inplace=True, axis='rows')

# Inspecting dataset
display(filtered[filtered.Month.isnull()])

,District,Type,TypeCause,DateAlert,DateExtinction,AreaTotal,FireDuration,DayAlert,Month,HourTotal,DayOfYear,HourOfDay
811,Porto,Florestal,Negligente,2015-03-06 11:37:00,NaT,3.797,NaT,Friday,NaN,NaN,65,11
843,Porto,Florestal,NaN,2015-07-20 13:08:00,NaT,2.000,NaT,Monday,NaN,NaN,201,13
846,Vila Real,Florestal,Reacendimento,2015-07-20 22:20:00,NaT,0.001,NaT,Monday,NaN,NaN,201,22
3248,Porto,Florestal,Negligente,2015-05-18 21:24:00,NaT,0.005,NaT,Monday,NaN,NaN,138,21
3323,Porto,Florestal,NaN,2015-06-27 00:05:00,NaT,0.001,NaT,Saturday,NaN,NaN,178,0
3901,Vila Real,Florestal,Negligente,2015-10-01 12:19:00,NaT,2.000,NaT,Thursday,NaN,NaN,274,12
3976,Vila Real,Florestal,Negligente,2015-09-29 13:18:00,NaT,0.050,NaT,Tuesday,NaN,NaN,272,13
4429,Porto,Florestal,NaN,2015-07-23 23:55:00,NaT,0.001,NaT,Thursday,NaN,NaN,204,23
4508,Porto,Agrícola,Desconhecida,2015-07-23 23:40:00,NaT,0.020,NaT,Thursday,NaN,NaN,204,23


,District,Type,TypeCause,DateAlert,DateExtinction,AreaTotal,FireDuration,DayAlert,Month,HourTotal,DayOfYear,HourOfDay


In [85]:
# Inspecting new dataset shape
print(f'The dataset has {filtered.shape[0]} rows and {filtered.shape[1]} columns. ')
print(f'Filtered dataframe accounts for {round(filtered.shape[0]/df.shape[0]*100, 2)}% of source')

The dataset has 5315 rows and 12 columns. 
Filtered dataframe accounts for 22.93% of source


In [87]:
# Inspecting whether the date of Alert is greater that Extinction
display(filtered[(filtered.DateAlert > filtered.DateExtinction) == True])

# Removing rows where date of Alert is greater that Extinction
north = filtered[(filtered.DateAlert > filtered.DateExtinction) == False].copy(deep=True)
display(north[(north.DateAlert > north.DateExtinction) == True])

,District,Type,TypeCause,DateAlert,DateExtinction,AreaTotal,FireDuration,DayAlert,Month,HourTotal,DayOfYear,HourOfDay
77,Vila Real,Florestal,Intencional,2015-03-06 16:39:00,2015-03-06 00:00:00,6.00,-1 days +07:21:00,Friday,Mar,-16.650000,65,16
689,Vila Real,Florestal,Negligente,2015-05-17 11:52:00,2015-05-17 00:23:00,21.00,-1 days +12:31:00,Sunday,May,-11.483333,137,11
2917,Vila Real,Florestal,NaN,2015-08-15 20:56:00,2015-08-15 02:20:00,2.50,-1 days +05:24:00,Saturday,Aug,-18.600000,227,20
3388,Vila Real,Florestal,Negligente,2015-07-21 19:09:00,2015-07-21 02:05:00,1.50,-1 days +06:56:00,Tuesday,Jul,-17.066667,202,19
3625,Vila Real,Florestal,Desconhecida,2015-07-11 20:43:00,2015-07-11 02:30:00,0.14,-1 days +05:47:00,Saturday,Jul,-18.216667,192,20


,District,Type,TypeCause,DateAlert,DateExtinction,AreaTotal,FireDuration,DayAlert,Month,HourTotal,DayOfYear,HourOfDay


In [88]:
north.head()

,District,Type,TypeCause,DateAlert,DateExtinction,AreaTotal,FireDuration,DayAlert,Month,HourTotal,DayOfYear,HourOfDay
0,Porto,Florestal,Negligente,2015-03-24 17:10:00,2015-03-24 18:47:00,1.35,0 days 01:37:00,Tuesday,Mar,1.616667,83,17
1,Vila Real,Florestal,Negligente,2015-03-24 21:40:00,2015-03-25 05:45:00,38.50,0 days 08:05:00,Tuesday,Mar,8.083333,83,21
2,Porto,Florestal,NaN,2015-03-25 15:30:00,2015-03-25 18:25:00,0.05,0 days 02:55:00,Wednesday,Mar,2.916667,84,15
3,Vila Real,Florestal,Negligente,2015-03-25 16:00:00,2015-03-25 17:00:00,0.01,0 days 01:00:00,Wednesday,Mar,1.000000,84,16
4,Vila Real,Florestal,Negligente,2015-03-25 16:40:00,2015-03-25 18:00:00,0.02,0 days 01:20:00,Wednesday,Mar,1.333333,84,16


In [89]:
print("TypeCause  Numbers")
display(north.TypeCause.value_counts(dropna=False, ascending=False, normalize=True))
print("Type Numbers")
display(north.Type.value_counts(dropna=False, ascending=False, normalize=True))


TypeCause  Numbers


NaN              0.391149
Negligente       0.247458
Desconhecida     0.140490
Intencional      0.114878
Reacendimento    0.103955
Natural          0.002072
Name: TypeCause, dtype: float64

Type Numbers


Florestal    0.950282
Agrícola     0.046516
Queimada     0.003202
Name: Type, dtype: float64

In [94]:
# show statistics values for all dataset
north.describe(include='all', datetime_is_numeric=True)

,District,Type,TypeCause,DateAlert,DateExtinction,AreaTotal,FireDuration,DayAlert,Month,HourTotal,DayOfYear,HourOfDay
count,5310,5310,3233,5310,5310,5310.000000,5310,5310,5310,5310.000000,5310.000000,5310.000000
unique,2,3,5,NaN,NaN,NaN,NaN,7,12,NaN,NaN,NaN
top,Porto,Florestal,Negligente,NaN,NaN,NaN,NaN,Sunday,Aug,NaN,NaN,NaN
freq,3989,5046,1314,NaN,NaN,NaN,NaN,895,1208,NaN,NaN,NaN
mean,NaN,NaN,NaN,2015-06-28 17:59:51.446327808,2015-06-28 20:05:35.898305280,1.530223,0 days 02:05:44.451977401,NaN,NaN,2.095681,179.132957,14.321846
min,NaN,NaN,NaN,2015-01-01 16:12:00,2015-01-01 17:20:00,0.000100,0 days 00:00:00,NaN,NaN,0.000000,1.000000,0.000000
25%,NaN,NaN,NaN,2015-05-19 18:14:15,2015-05-19 19:53:45,0.010000,0 days 01:01:00,NaN,NaN,1.016667,139.000000,12.000000
50%,NaN,NaN,NaN,2015-07-13 17:20:30,2015-07-13 19:35:00,0.050000,0 days 01:34:00,NaN,NaN,1.566667,194.000000,15.000000
75%,NaN,NaN,NaN,2015-08-10 18:50:45,2015-08-10 20:30:00,0.400000,0 days 02:25:45,NaN,NaN,2.429167,222.000000,18.000000
max,NaN,NaN,NaN,2015-12-27 13:55:00,2015-12-27 18:45:00,494.000000,6 days 02:58:00,NaN,NaN,146.966667,361.000000,23.000000


In [ ]:
g = sns.FacetGrid(filtered, col="District", height= 12, hue="Type")
g.map_dataframe(sns.histplot, x="Type")
g.set_axis_labels("", "Frequency")
g.set_titles(col_template="{col_name} District", row_template="{row_name}")
g.fig.suptitle("Type of Fire By District")
g.tight_layout();

In [ ]:
g = sns.FacetGrid(filtered, col="District", height= 12, hue="TypeCause")
g.map_dataframe(sns.histplot, x="TypeCause")
g.set_axis_labels("", "Frequency")
g.set_titles(col_template="{col_name} District", row_template="{row_name}")
g.fig.suptitle("TypeCause of Fire By District")
g.tight_layout();

In [ ]:
g = sns.FacetGrid(filtered, col="District", height= 12, hue="HourOfDay")
g.map_dataframe(sns.histplot, x="HourOfDay")
g.set_axis_labels("", "Frequency")
g.set_titles(col_template="{col_name} District", row_template="{row_name}")
g.fig.suptitle("Hour of Fire By District")
g.set(xticks=np.arange(0,24,1))
g.set_xticklabels(np.arange(0,24,1))
g.tight_layout();

In [ ]:
g = sns.FacetGrid(filtered, col="District", height= 12, hue="DayAlert")
g.map_dataframe(sns.histplot, x="DayAlert")
g.set_axis_labels("", "Frequency")
g.set_titles(col_template="{col_name} District", row_template="{row_name}")
g.fig.suptitle("DayAlert of Fire By District")

g.tight_layout();

In [ ]:
g = sns.FacetGrid(filtered, col="District", height= 12, hue="Month")
g.map_dataframe(sns.histplot, x="Month")
g.set_axis_labels("", "Frequency")
g.set_titles(col_template="{col_name} District", row_template="{row_name}")
g.fig.suptitle("DayAlert of Fire By District")

g.tight_layout();

In [ ]:
g = sns.FacetGrid(filtered, col="District", height= 12, hue="DayOfYear")
g.map_dataframe(sns.histplot, x="DayOfYear")
g.set_axis_labels("", "Frequency")
g.set_titles(col_template="{col_name} District", row_template="{row_name}")
g.fig.suptitle("DayOfYear of Fire By District")
g.set(xticks=np.arange(0,366,90))
g.set_xticklabels(np.arange(0,366,90))
g.tight_layout();

In [ ]:
burnArea = filtered.groupby('District')['AreaTotal'].agg('sum')

burnArea

In [ ]:
burnAreaType = filtered.query('AreaTotal > 0').groupby(['District','Type'])['AreaTotal'].sum().reset_index(name='totalArea')
burnAreaType

In [ ]:
burnArea = filtered.query('AreaTotal > 0').groupby('District')['AreaTotal'].sum().reset_index(name='totalArea')

sns.barplot(x="District", y='totalArea', data=burnArea);

In [ ]:
filtered.query('AreaTotal > 0')

In [ ]:
filtered.head()

In [ ]:
stats = filtered.query('AreaTotal > 0')
stats.head()

In [ ]:
filtered.groupby(['District','Type'])['AreaTotal'].sum()

In [ ]:
# group by stats for burned area group by distrito
filtered.query('AreaTotal > 0').groupby(by="District")['District'].size().describe()

In [ ]:
filtered.query('AreaTotal > 0').groupby('District')['Type'].size()

In [ ]:
aux = filtered.query('AreaTotal > 0').sort_values(by='HourTotal', ascending=False)

aux.Type.unique()

In [ ]:
# Relation between total burnt area and Fire Duration
sns.scatterplot(data=aux, x='AreaTotal', y='HourTotal', hue='District')
# set labels
plt.xlabel("Area Burned (ha)", size=15)
plt.ylabel("Wildfire Duration", size=15)
plt.title("Sort Bars in Barplot in Descending Order", size=18)
plt.tight_layout()

In [ ]:
aux.head()

In [ ]:
aux[aux.HourTotal.isna()]

In [ ]:
aux[aux.HourTotal < 1]
